<a href="https://colab.research.google.com/github/prof-rossetti/intro-to-python/blob/main/exercises/csv-processing/Stock_Data_Processing_with_Pandas_(Summer_2023).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will focus on using the `pandas` package for practical processing of financial data.

In Part 1, we will demonstrate our ability to fetch CSV formatted stock price data from the [AlphaVantage API](https://www.alphavantage.co/). We have already practiced fetching JSON formatted data from this API, however some of the endpoints offer the data in CSV format, which will most likely be easier to process.

In Part 2, we will demonstrate our ability to fetch stock price data from the `YahooQuery` package. We will see `YahooQuery` makes it easy to get stock prices for multiple companies at once. Then with this data about multiple companies, we will focus on comparing the growth rates of each company, and determining which company has the highest growth.

## 1) Data from AlphaVantage API



We can use the `pandas.read_csv` function to process dynamic CSV data provided by an API. For this example, let's first obtain and supply an AlphaVantage API Key, or use one of the prof's premium keys:

In [ ]:
from getpass import getpass

API_KEY = getpass("Please input your API key:") or "demo"

Please input your API key:··········


In [ ]:
#from pandas import read_csv
#
#read_csv("daily_adjusted_GOOGL.csv")

In [ ]:
from pandas import read_csv

# see: https://www.alphavantage.co/documentation/#dailyadj
# ... the datatype=csv URL param gives us data in CSV format, which may be easier to work with

# request_url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=demo&datatype=json"

symbol = "MSFT"
request_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={API_KEY}&datatype=csv"

prices_df = read_csv(request_url)
prices_df.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2023-06-21,336.37,337.73,332.07,333.56,333.56,24865845,0.0,1.0
1,2023-06-20,339.31,342.08,335.86,338.05,338.05,26375407,0.0,1.0
2,2023-06-16,351.32,351.47,341.95,342.33,342.33,46551985,0.0,1.0
3,2023-06-15,337.48,349.84,337.20,348.10,348.10,38899075,0.0,1.0
4,2023-06-14,334.34,339.04,332.81,337.34,337.34,26003791,0.0,1.0


In [ ]:
print(type(prices_df))

<class 'pandas.core.frame.DataFrame'>


#### Getting the Latest Closing Price

In [ ]:
# column centric approach
prices_df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume',
       'dividend_amount', 'split_coefficient'],
      dtype='object')

In [ ]:
print(type(prices_df["adjusted_close"]))
prices_df["adjusted_close"]

<class 'pandas.core.series.Series'>


0     333.560000
1     338.050000
2     342.330000
3     348.100000
4     337.340000
         ...    
95    263.363259
96    251.568646
97    246.651736
98    241.575573
99    247.000100
Name: adjusted_close, Length: 100, dtype: float64

In [ ]:
prices_df["adjusted_close"][0]

333.56

In [ ]:
# row centric approach

prices_df.iloc[0]

timestamp            2023-06-21
open                     336.37
high                     337.73
low                      332.07
close                    333.56
adjusted_close           333.56
volume                 24865845
dividend_amount             0.0
split_coefficient           1.0
Name: 0, dtype: object

In [ ]:
prices_df.iloc[0]["adjusted_close"]

333.56

In [ ]:
# example mapping operation
prices_df["adjusted_close"] * 100

0     33356.000000
1     33805.000000
2     34233.000000
3     34810.000000
4     33734.000000
          ...     
95    26336.325918
96    25156.864610
97    24665.173567
98    24157.557308
99    24700.009977
Name: adjusted_close, Length: 100, dtype: float64

In [ ]:
# quick check for stock splits, because they might warrant adjustments to the other columns as well
prices_df["split_coefficient"].value_counts() #> all values are 1.0, so in this case we have no splits

1.0    100
Name: split_coefficient, dtype: int64

Now that we have our prices as a pandas series (or column), aggregations are easy. Let's calculate the average price, the 100 day high and low, as well as the volatility (standard deviation) of prices.

In [ ]:
print("AVERAGE CLOSING PRICE:")
prices_df["adjusted_close"].mean()

AVERAGE CLOSING PRICE:


288.36124891224006

In [ ]:
# FYI - if there were stock splits, we would have to create an adjusted version of the high
print("100 DAY HIGH PRICE:")
print(prices_df["high"].max())

100 DAY HIGH PRICE:
351.47


In [ ]:
# FYI - if there were stock splits, we would have to create an adjusted version of the low
print("100 DAY LOW PRICE:")
print(prices_df["low"].min())

100 DAY LOW PRICE:
242.2


In [ ]:
print("VOLATILITY OF CLOSING PRICES (STANDARD DEVIATION):")
prices_df["adjusted_close"].std()

VOLATILITY OF CLOSING PRICES (STANDARD DEVIATION):


29.394966643762185

### Plotting Stock Prices

In [ ]:
import plotly.express as px

#px.line(x=[1,2,3], y=[100, 200, 300], title=f"Adjusted Closing Prices for {symbol}")

#px.line(x=prices_df["timestamp"], y=prices_df["adjusted_close"], title=f"Adjusted Closing Prices for {symbol}")

In [ ]:
import plotly.express as px

px.line(prices_df, x="timestamp", y="adjusted_close", title=f"Adjusted Closing Prices for {symbol}")

## 2) Data from `YahooQuery` Package

In [ ]:
# setup cell (run and leave as is)

%%capture

!pip install yahooquery

https://yahooquery.dpguthrie.com/guide/ticker/intro/

In [ ]:
# setup cell (run and leave as is)

from yahooquery import Ticker

symbol = "MSFT"
company = Ticker(symbol)
type(company)

yahooquery.ticker.Ticker

### Exploring Available Datasets

In [ ]:
# dir(company)

In [ ]:
# company.country #> str
# company.asset_profile #> dict
# company.calendar_events #> dict
# company.earnings #> dict
# company.earnings_trend #> dict
# company.esg_scores #> dict
# company.financial_data #> dict
# company.key_stats #> dict
# company.price #> dict
# company.recommendations #> dict of similar stocks and reco scores
# company.summary_detail #> dict
# company.summary_profile #> dict with address and long summary and number of employees
# company.technical_insights #> dict

# company.dividend_history(start="2020-01-01") #> series or df

# company.all_financial_data() #> dataframe
# company.balance_sheet() #> dataframe
# company.cash_flow() #> dataframe
# company.company_officers #> dataframe
# company.corporate_events #> dataframe
# company.corporate_guidance #> dataframe
# company.earning_history #> dataframe
# company.history() #> dataframe of daily prices
# company.income_statement() #> dataframe
# company.news() #> ["error"]
# company.option_chain #> dataframe
# company.recommendation_trend #> df row per period
# company.sec_filings #> dataframe

Dividend history:

In [ ]:
company.dividend_history(start="2021-01-01").head()

dividends
symbol date                 
MSFT   2021-02-17       0.56
       2021-05-19       0.56
       2021-08-18       0.56
       2021-11-17       0.62
       2022-02-16       0.62

In [ ]:
company.all_financial_data()

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,...,TotalNonCurrentLiabilitiesNetMinorityInterest,TotalOperatingIncomeAsReported,TotalRevenue,TotalTaxPayable,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,TradeandOtherPayablesNonCurrent,WorkInProcess,WorkingCapital,WriteOff
symbol,,,,,,,,,,,,,,,,,,,,,
MSFT,2019-06-30,12M,USD,9.382000e+09,2.952400e+10,-3.533000e+10,-411000000.0,7.673000e+09,5.11,1.194600e+10,...,1.148060e+11,4.295900e+10,1.258430e+11,5.665000e+09,7.100000e+08,7.100000e+08,2.961200e+10,53000000.0,1.061320e+11,NaN
MSFT,2020-06-30,12M,USD,1.253000e+10,3.201100e+10,-4.319700e+10,-788000000.0,7.610000e+09,5.82,1.135600e+10,...,1.106970e+11,5.295900e+10,1.430150e+11,2.130000e+09,2.800000e+07,2.800000e+07,2.943200e+10,83000000.0,1.096050e+11,NaN
MSFT,2021-06-30,12M,USD,1.516300e+10,3.804300e+10,-5.135100e+10,-751000000.0,7.547000e+09,8.12,1.357600e+10,...,1.031340e+11,6.991600e+10,1.680880e+11,2.174000e+09,1.303000e+09,1.303000e+09,2.719000e+10,79000000.0,9.574900e+10,NaN
MSFT,2022-06-30,12M,USD,1.900000e+10,4.426100e+10,-5.966000e+10,-633000000.0,7.496000e+09,9.70,1.422400e+10,...,1.032160e+11,8.338300e+10,1.982700e+11,4.067000e+09,3.340000e+08,3.340000e+08,2.606900e+10,82000000.0,7.460200e+10,101000000.0


In [ ]:
company.balance_sheet() #> dataframe

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AllowanceForDoubtfulAccountsReceivable,BuildingsAndImprovements,CapitalLeaseObligations,CapitalStock,...,TotalCapitalization,TotalDebt,TotalEquityGrossMinorityInterest,TotalLiabilitiesNetMinorityInterest,TotalNonCurrentAssets,TotalNonCurrentLiabilitiesNetMinorityInterest,TotalTaxPayable,TradeandOtherPayablesNonCurrent,WorkInProcess,WorkingCapital
symbol,,,,,,,,,,,,,,,,,,,,,
MSFT,2019-06-30,12M,USD,9.382000e+09,2.952400e+10,-3.533000e+10,-411000000.0,2.628800e+10,6.188000e+09,7.852000e+10,...,1.689920e+11,7.836600e+10,1.023300e+11,1.842260e+11,1.110040e+11,1.148060e+11,5.665000e+09,2.961200e+10,53000000.0,1.061320e+11
MSFT,2020-06-30,12M,USD,1.253000e+10,3.201100e+10,-4.319700e+10,-788000000.0,3.399500e+10,7.671000e+09,8.055200e+10,...,1.778820e+11,7.099800e+10,1.183040e+11,1.830070e+11,1.193960e+11,1.106970e+11,2.130000e+09,2.943200e+10,83000000.0,1.096050e+11
MSFT,2021-06-30,12M,USD,1.516300e+10,3.804300e+10,-5.135100e+10,-751000000.0,4.392800e+10,9.629000e+09,8.311100e+10,...,1.920620e+11,6.777500e+10,1.419880e+11,1.917910e+11,1.493730e+11,1.031340e+11,2.174000e+09,2.719000e+10,79000000.0,9.574900e+10
MSFT,2022-06-30,12M,USD,1.900000e+10,4.426100e+10,-5.966000e+10,-633000000.0,5.501400e+10,1.148900e+10,8.693900e+10,...,2.135740e+11,6.127000e+10,1.665420e+11,1.982980e+11,1.951560e+11,1.032160e+11,4.067000e+09,2.606900e+10,82000000.0,7.460200e+10


In [ ]:
company.cash_flow()

,asOfDate,periodType,currencyCode,BeginningCashPosition,CapitalExpenditure,CashDividendsPaid,CashFlowFromContinuingFinancingActivities,CashFlowFromContinuingInvestingActivities,CashFlowFromContinuingOperatingActivities,ChangeInAccountPayable,...,NetShortTermDebtIssuance,OperatingCashFlow,OperatingGainsLosses,PurchaseOfBusiness,PurchaseOfInvestment,PurchaseOfPPE,RepaymentOfDebt,RepurchaseOfCapitalStock,SaleOfInvestment,StockBasedCompensation
symbol,,,,,,,,,,,,,,,,,,,,,
MSFT,2019-06-30,12M,USD,1.194600e+10,-1.392500e+10,-1.381100e+10,-3.688700e+10,-1.577300e+10,5.218500e+10,2.320000e+08,...,0.0,5.218500e+10,-7.920000e+08,-2.388000e+09,-5.769700e+10,-1.392500e+10,-4.000000e+09,-1.954300e+10,5.823700e+10,4.652000e+09
MSFT,2020-06-30,12M,USD,1.135600e+10,-1.544100e+10,-1.513700e+10,-4.603100e+10,-1.222300e+10,6.067500e+10,3.018000e+09,...,0.0,6.067500e+10,-2.190000e+08,-2.521000e+09,-7.719000e+10,-1.544100e+10,-5.518000e+09,-2.296800e+10,8.417000e+10,5.289000e+09
MSFT,2021-06-30,12M,USD,1.357600e+10,-2.062200e+10,-1.652100e+10,-4.848600e+10,-2.757700e+10,7.674000e+10,2.798000e+09,...,NaN,7.674000e+10,-1.249000e+09,-8.909000e+09,-6.292400e+10,-2.062200e+10,-3.750000e+09,-2.738500e+10,6.580000e+10,6.118000e+09
MSFT,2022-06-30,12M,USD,1.422400e+10,-2.388600e+10,-1.813500e+10,-5.887600e+10,-3.031100e+10,8.903500e+10,2.943000e+09,...,NaN,8.903500e+10,-4.090000e+08,-2.203800e+10,-2.645600e+10,-2.388600e+10,-9.023000e+09,-3.269600e+10,4.489400e+10,7.502000e+09
MSFT,2022-12-31,TTM,USD,2.060400e+10,-2.476800e+10,-1.896400e+10,-5.284600e+10,-3.618200e+10,8.438600e+10,-4.460000e+08,...,NaN,8.438600e+10,4.540000e+08,-2.101000e+10,-3.025400e+10,-2.476800e+10,-5.947000e+09,-2.861100e+10,4.333000e+10,8.633000e+09
MSFT,2023-03-31,TTM,USD,1.249800e+10,-2.603500e+10,-1.937800e+10,-4.579100e+10,-2.327500e+10,8.344100e+10,-1.373000e+09,...,NaN,8.344100e+10,3.090000e+08,-2.592000e+09,-3.059400e+10,-2.603500e+10,-1.750000e+09,-2.529800e+10,3.993100e+10,9.192000e+09


In [ ]:
company.company_officers

maxAge                          name   age  \
symbol row                                               
MSFT   0         1            Mr. Satya  Nadella  55.0   
       1         1     Mr. Bradford L. Smith LCA  63.0   
       2         1               Ms. Amy E. Hood  50.0   
       3         1           Mr. Judson  Althoff  49.0   
       4         1   Mr. Christopher David Young  50.0   
       5         1            Ms. Alice L. Jolla  55.0   
       6         1         Mr. James Kevin Scott  50.0   
       7         1                Brett  Iversen   NaN   
       8         1             Mr. Frank X. Shaw   NaN   
       9         1  Mr. Christopher C. Capossela  52.0   

                                                   title  yearBorn  \
symbol row                                                           
MSFT   0                                  Chairman & CEO    1967.0   
       1                            Pres & Vice Chairman    1959.0   
       2                                  Exec. VP & CFO    1972.0   
       3             Exec. VP & Chief Commercial Officer    1973.0   
       4    Exec. VP of Bus. Devel., Strategy & Ventures    1972.0   
       5             Corp. VP & Chief Accounting Officer    1967.0   
       6                            Exec. VP of AI & CTO    1972.0   
       7                Gen. Mang. of Investor Relations       NaN   
       8               Corp. VP for Corp. Communications       NaN   
       9              Exec. VP & Chief Marketing Officer    1970.0   

            fiscalYear    totalPay  exercisedValue  unexercisedValue  
symbol row                                                            
MSFT   0        2022.0  12676750.0               0                 0  
       1        2022.0   4655274.0               0                 0  
       2        2022.0   4637915.0               0                 0  
       3        2022.0   4428268.0               0                 0  
       4        2022.0   4588876.0               0                 0  
       5           NaN         NaN               0                 0  
       6           NaN         NaN               0                 0  
       7           NaN         NaN               0                 0  
       8           NaN         NaN               0                 0  
       9           NaN         NaN               0                 0

In [ ]:
company.corporate_events


id significance  \
symbol date                               
MSFT   2013-09-17  2832441            1   
       2013-11-20  2872539            3   
       2014-03-11  2942626            3   
       2014-06-10  3007564            3   
       2014-09-16  3069314            1   
...                    ...          ...   
       2023-01-24  4649139            1   
       2023-04-25  4695510            1   
       2023-04-25  4695644            1   
       2023-04-26  4696423            1   
       2023-04-28  4699419            1   

                                                            headline  \
symbol date                                                            
MSFT   2013-09-17  Microsoft Corp Announces Quarterly Dividend In...   
       2013-11-20        Microsoft Corp Declares Quarterly Dividend    
       2014-03-11         Microsoft Corp declares quarterly dividend   
       2014-06-10         Microsoft Corp declares quarterly dividend   
       2014-09-16  Microsoft Corp announces quarterly dividend in...   
...                                                              ...   
       2023-01-24  Microsoft Sees Q3 Productivity And Business Pr...   
       2023-04-25  Microsoft Reports Quarterly Revenue In More Pe...   
       2023-04-25  Microsoft Says Sees Q4 Intelligent Cloud Reven...   
       2023-04-26  Activision Blizzard Announces First Quarter 20...   
       2023-04-28  Meta Is Talking To Microsoft/OpenAI About Doin...   

                                                         description  \
symbol date                                                            
MSFT   2013-09-17  Microsoft Corp. announced on that its Board of...   
       2013-11-20  Microsoft Corp announced that its Board of Dir...   
       2014-03-11  Microsoft Corp:Declares quarterly dividend of ...   
       2014-06-10  Microsoft Corp:Says that its board declared qu...   
       2014-09-16  Microsoft Corp:Says that its board declared qu...   
...                                                              ...   
       2023-01-24  Jan 24 (Reuters) - Microsoft Corp <MSFT.O>::MI...   
       2023-04-25  April 25 (Reuters) - Microsoft Corp <MSFT.O>::...   
       2023-04-25  April 25 (Reuters) - Microsoft Corp <MSFT.O>::...   
       2023-04-26  April 26 (Reuters) - Activision Blizzard Inc <...   
       2023-04-28  April 28 (Reuters) - :META IS TALKING TO MICRO...   

                         parentTopics  
symbol date                            
MSFT   2013-09-17         Performance  
       2013-11-20         Performance  
       2014-03-11         Performance  
       2014-06-10         Performance  
       2014-09-16         Performance  
...                               ...  
       2023-01-24  Corporate Guidance  
       2023-04-25  Corporate Guidance  
       2023-04-25  Corporate Guidance  
       2023-04-26  Corporate Guidance  
       2023-04-28  Corporate Guidance  

[170 rows x 5 columns]

In [ ]:
company.corporate_guidance

id significance  \
symbol date                               
MSFT   2014-07-17  3031584            1   
       2016-05-24  3380603            1   
       2016-06-13  3390319            1   
       2016-07-19  3407917            1   
       2016-07-19  3407928            1   
...                    ...          ...   
       2023-01-24  4649139            1   
       2023-04-25  4695510            1   
       2023-04-25  4695644            1   
       2023-04-26  4696423            1   
       2023-04-28  4699419            1   

                                                            headline  \
symbol date                                                            
MSFT   2014-07-17  Microsoft announces steps to simplify organiza...   
       2016-05-24  Finland's Kone picks CFO from Microsoft's phon...   
       2016-06-13  Microsoft sees $150 mln of cost synergies annu...   
       2016-07-19  Microsoft sees negative forex impact of 2 pts ...   
       2016-07-19  Microsoft Corp CFO says expect full year opera...   
...                                                              ...   
       2023-01-24  Microsoft Sees Q3 Productivity And Business Pr...   
       2023-04-25  Microsoft Reports Quarterly Revenue In More Pe...   
       2023-04-25  Microsoft Says Sees Q4 Intelligent Cloud Reven...   
       2023-04-26  Activision Blizzard Announces First Quarter 20...   
       2023-04-28  Meta Is Talking To Microsoft/OpenAI About Doin...   

                                                         description  \
symbol date                                                            
MSFT   2014-07-17  Microsoft Corp:Says restructuring plan to simp...   
       2016-05-24  Kone Oyj <KNEBV.HE> : says appoints Ilkka Hara...   
       2016-06-13  Microsoft Corp <MSFT.O>: Sees $150 mln of cost...   
       2016-07-19  Microsoft Corp <MSFT.O> : Sees negative forex ...   
       2016-07-19  Microsoft Corp CFO Hood : In q4 we did see add...   
...                                                              ...   
       2023-01-24  Jan 24 (Reuters) - Microsoft Corp <MSFT.O>::MI...   
       2023-04-25  April 25 (Reuters) - Microsoft Corp <MSFT.O>::...   
       2023-04-25  April 25 (Reuters) - Microsoft Corp <MSFT.O>::...   
       2023-04-26  April 26 (Reuters) - Activision Blizzard Inc <...   
       2023-04-28  April 28 (Reuters) - :META IS TALKING TO MICRO...   

                                           parentTopics  
symbol date                                              
MSFT   2014-07-17  Restructuring/Reorganization/Related  
       2016-05-24                           Performance  
       2016-06-13                           Performance  
       2016-07-19                    Corporate Guidance  
       2016-07-19                    Corporate Guidance  
...                                                 ...  
       2023-01-24                    Corporate Guidance  
       2023-04-25                    Corporate Guidance  
       2023-04-25                    Corporate Guidance  
       2023-04-26                    Corporate Guidance  
       2023-04-28                    Corporate Guidance  

[76 rows x 5 columns]

In [ ]:
company.earning_history

maxAge  epsActual  epsEstimate  epsDifference  surprisePercent  \
symbol row                                                                   
MSFT   0         1       2.23         2.29          -0.06           -0.026   
       1         1       2.35         2.30           0.05            0.022   
       2         1       2.32         2.29           0.03            0.013   
       3         1       2.45         2.23           0.22            0.099   

               quarter period  
symbol row                     
MSFT   0    2022-06-30    -4q  
       1    2022-09-30    -3q  
       2    2022-12-31    -2q  
       3    2023-03-31    -1q

Looks like this "history" dataframe has the historical prices, which are probably of particular interest:

In [ ]:
company.history()

open        high         low       close    volume  \
symbol date                                                                   
MSFT   2023-01-03  243.080002  245.750000  237.399994  239.580002  25740000   
       2023-01-04  232.279999  232.869995  225.960007  229.100006  50623400   
       2023-01-05  227.199997  227.550003  221.759995  222.309998  39585600   
       2023-01-06  223.000000  225.759995  219.350006  224.929993  43613600   
       2023-01-09  226.449997  231.240005  226.410004  227.119995  27369800   
...                       ...         ...         ...         ...       ...   
       2023-06-14  334.339996  339.040009  332.809998  337.339996  26003800   
       2023-06-15  337.480011  349.839996  337.200012  348.100006  38899100   
       2023-06-16  351.320007  351.470001  341.950012  342.329987  46533600   
       2023-06-20  339.309998  342.079987  335.859985  338.049988  26375400   
       2023-06-21  336.369995  337.730011  332.070007  333.559998  25090600   

                     adjclose  dividends  
symbol date                               
MSFT   2023-01-03  238.460129        0.0  
       2023-01-04  228.029114        0.0  
       2023-01-05  221.270859        0.0  
       2023-01-06  223.878616        0.0  
       2023-01-09  226.058365        0.0  
...                       ...        ...  
       2023-06-14  337.339996        0.0  
       2023-06-15  348.100006        0.0  
       2023-06-16  342.329987        0.0  
       2023-06-20  338.049988        0.0  
       2023-06-21  333.559998        0.0  

[117 rows x 7 columns]

In [ ]:
company.income_statement()

,asOfDate,periodType,currencyCode,BasicAverageShares,BasicEPS,CostOfRevenue,DilutedAverageShares,DilutedEPS,DilutedNIAvailtoComStockholders,EBIT,...,SpecialIncomeCharges,TaxEffectOfUnusualItems,TaxProvision,TaxRateForCalcs,TotalExpenses,TotalOperatingIncomeAsReported,TotalRevenue,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,WriteOff
symbol,,,,,,,,,,,,,,,,,,,,,
MSFT,2019-06-30,12M,USD,7.673000e+09,5.11,4.291000e+10,7.753000e+09,5.06,3.924000e+10,4.295900e+10,...,0.0,0.000000e+00,4.448000e+09,0.100000,8.288400e+10,4.295900e+10,1.258430e+11,7.100000e+08,7.100000e+08,NaN
MSFT,2020-06-30,12M,USD,7.610000e+09,5.82,4.607800e+10,7.683000e+09,5.76,4.428100e+10,5.295900e+10,...,NaN,0.000000e+00,8.755000e+09,0.160000,9.005600e+10,5.295900e+10,1.430150e+11,2.800000e+07,2.800000e+07,NaN
MSFT,2021-06-30,12M,USD,7.547000e+09,8.12,5.223200e+10,7.608000e+09,8.05,6.127100e+10,6.991600e+10,...,NaN,0.000000e+00,9.831000e+09,0.138266,9.817200e+10,6.991600e+10,1.680880e+11,1.303000e+09,1.303000e+09,NaN
MSFT,2022-06-30,12M,USD,7.496000e+09,9.70,6.265000e+10,7.540000e+09,9.65,7.273800e+10,8.338300e+10,...,-101000000.0,0.000000e+00,1.097800e+10,0.131134,1.148870e+11,8.338300e+10,1.982700e+11,3.340000e+08,3.340000e+08,101000000.0
MSFT,2022-12-31,TTM,USD,NaN,NaN,6.498400e+10,NaN,NaN,6.744900e+10,8.457700e+10,...,NaN,-1.002692e+08,1.513900e+10,0.183308,1.212790e+11,8.281500e+10,2.040940e+11,-5.470000e+08,-5.470000e+08,NaN
MSFT,2023-03-31,TTM,USD,NaN,NaN,6.549700e+10,NaN,NaN,6.902000e+10,8.705300e+10,...,NaN,-3.886760e+07,1.605100e+10,0.188678,1.227880e+11,8.480300e+10,2.075910e+11,-2.060000e+08,-2.060000e+08,NaN


In [ ]:
#company.news() #> ["error"]

In [ ]:
company.option_chain

contractSymbol  strike currency  lastPrice  \
symbol expiration optionType                                                    
MSFT   2023-06-23 calls       MSFT230623C00215000   215.0      USD     122.57   
                  calls       MSFT230623C00220000   220.0      USD     124.70   
                  calls       MSFT230623C00225000   225.0      USD     115.80   
                  calls       MSFT230623C00230000   230.0      USD     104.15   
                  calls       MSFT230623C00235000   235.0      USD      92.09   
...                                           ...     ...      ...        ...   
       2025-12-19 puts        MSFT251219P00420000   420.0      USD      94.15   
                  puts        MSFT251219P00430000   430.0      USD     133.00   
                  puts        MSFT251219P00440000   440.0      USD     122.15   
                  puts        MSFT251219P00450000   450.0      USD     108.49   
                  puts        MSFT251219P00460000   460.0      USD     150.94   

                                change  percentChange  volume  openInterest  \
symbol expiration optionType                                                  
MSFT   2023-06-23 calls       0.000000       0.000000     3.0             3   
                  calls       0.000000       0.000000     5.0             5   
                  calls       0.000000       0.000000     3.0             4   
                  calls      -4.470001      -4.115265     2.0            22   
                  calls       0.000000       0.000000     1.0            35   
...                                ...            ...     ...           ...   
       2025-12-19 puts       -4.949997      -4.994952     1.0            33   
                  puts        0.000000       0.000000     0.0             1   
                  puts        0.000000       0.000000     1.0             1   
                  puts        0.000000       0.000000     0.0             2   
                  puts        0.000000       0.000000     2.0             0   

                                 bid     ask contractSize       lastTradeDate  \
symbol expiration optionType                                                    
MSFT   2023-06-23 calls       117.40  120.20      REGULAR 2023-06-15 13:38:58   
                  calls       112.35  115.15      REGULAR 2023-06-16 18:52:31   
                  calls       107.35  110.05      REGULAR 2023-06-15 13:45:12   
                  calls       102.55  105.15      REGULAR 2023-06-21 14:47:10   
                  calls        97.45  100.30      REGULAR 2023-06-09 16:54:15   
...                              ...     ...          ...                 ...   
       2025-12-19 puts         92.20   96.05      REGULAR 2023-06-21 15:34:48   
                  puts        100.35  104.20      REGULAR 2023-04-26 16:58:59   
                  puts        100.50  105.00      REGULAR 2023-05-19 13:41:20   
                  puts        116.30  120.50      REGULAR 2023-06-15 17:20:56   
                  puts        121.75  125.50      REGULAR 2023-05-15 17:13:05   

                              impliedVolatility  inTheMoney  
symbol expiration optionType                                 
MSFT   2023-06-23 calls                2.667972        True  
                  calls                2.464848        True  
                  calls                2.253911        True  
                  calls                2.361332        True  
                  calls                2.267582        True  
...                                         ...         ...  
       2025-12-19 puts                 0.162759        True  
                  puts                 0.159829        True  
                  puts                 0.000010        True  
                  puts                 0.146844        True  
                  puts                 0.000010        True  

[2234 rows x 14 columns]

In [ ]:
company.recommendation_trend

period  strongBuy  buy  hold  sell  strongSell
symbol row                                               
MSFT   0       0m         14   13     6     0           1
       1      -1m         20   26     8     0           1
       2      -2m         19   26     8     0           1
       3      -3m         19   26     8     0           1

In [ ]:
company.sec_filings

date            epochDate      type  \
symbol row                                              
MSFT   0    2023-05-30  2023-05-30 00:00:00        SD   
       1    2023-04-25  2023-04-25 00:00:00       8-K   
       2    2023-03-24  2023-03-24 00:00:00      11-K   
       3    2023-02-09  2023-02-09 00:00:00  SC 13G/A   
       4    2023-01-31  2023-01-31 00:00:00  SC 13G/A   
       5    2023-01-24  2023-01-24 00:00:00      10-Q   
       6    2023-01-18  2023-01-18 00:00:00       8-K   
       7    2022-12-16  2022-12-16 00:00:00       8-K   
       8    2022-11-21  2022-11-21 00:00:00   PX14A6G   
       9    2022-11-04  2022-11-04 00:00:00   PX14A6G   
       10   2022-10-27  2022-10-27 00:00:00   DEFA14A   
       11   2022-10-25  2022-10-25 00:00:00      10-Q   
       12   2022-07-28  2022-07-28 00:00:00      10-K   
       13   2022-07-26  2022-07-26 00:00:00       8-K   
       14   2022-06-23  2022-06-23 00:00:00      11-K   
       15   2022-06-02  2022-06-02 00:00:00       8-K   
       16   2022-05-31  2022-05-31 00:00:00        SD   
       17   2022-04-26  2022-04-26 00:00:00      10-Q   
       18   2022-03-21  2022-03-21 00:00:00      11-K   
       19   2022-02-10  2022-02-10 00:00:00  SC 13G/A   
       20   2022-02-08  2022-02-08 00:00:00  SC 13G/A   
       21   2022-01-25  2022-01-25 00:00:00       8-K   

                                                        title  \
symbol row                                                      
MSFT   0    Specialized Disclosure Report filed pursuant t...   
       1                   Corporate Changes & Voting Matters   
       2             Annual report of employee stock purchase   
       3                     Tender Offer/Acquisition Reports   
       4                     Tender Offer/Acquisition Reports   
       5                           Periodic Financial Reports   
       6                   Corporate Changes & Voting Matters   
       7                   Corporate Changes & Voting Matters   
       8                        Notice of exempt solicitation   
       9                        Notice of exempt solicitation   
       10                                    Proxy Statements   
       11                          Periodic Financial Reports   
       12                          Periodic Financial Reports   
       13                  Corporate Changes & Voting Matters   
       14            Annual report of employee stock purchase   
       15                  Corporate Changes & Voting Matters   
       16   Specialized Disclosure Report filed pursuant t...   
       17                          Periodic Financial Reports   
       18            Annual report of employee stock purchase   
       19                    Tender Offer/Acquisition Reports   
       20                    Tender Offer/Acquisition Reports   
       21                  Corporate Changes & Voting Matters   

                                                     edgarUrl  \
symbol row                                                      
MSFT   0    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       1    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       2    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       3    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       4    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       5    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       6    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       7    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       8    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       9    https://finance.yahoo.com/sec-filing/MSFT/0001...   
       10   https://finance.yahoo.com/sec-filing/MSFT/0001...   
       11   https://finance.yahoo.com/sec-filing/MSFT/0001...   
       12   https://finance.yahoo.com/sec-filing/MSFT/0001...   
       13   https://finance.yahoo.com/sec-filing/MSFT/0001...   
       14   https://finance.yahoo.com/sec-filing

### Plotting Stock Prices

The `history` dataframe has the stock prices, but it looks like this dataframe has a multi-index, so we'll need to flatten it before charting.

https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.get_level_values.html


In [ ]:
history_df = company.history()
#history_df.index.get_level_values(0)
#history_df.index.get_level_values(1)

history_df["symbol"] = history_df.index.get_level_values(0)
history_df["date"] = history_df.index.get_level_values(1)
history_df.reset_index(drop=True, inplace=True) # reset the index for simplification
history_df

,open,high,low,close,volume,adjclose,dividends,symbol,date
0,243.080002,245.750000,237.399994,239.580002,25740000,238.460129,0.0,MSFT,2023-01-03
1,232.279999,232.869995,225.960007,229.100006,50623400,228.029114,0.0,MSFT,2023-01-04
2,227.199997,227.550003,221.759995,222.309998,39585600,221.270859,0.0,MSFT,2023-01-05
3,223.000000,225.759995,219.350006,224.929993,43613600,223.878616,0.0,MSFT,2023-01-06
4,226.449997,231.240005,226.410004,227.119995,27369800,226.058365,0.0,MSFT,2023-01-09
...,...,...,...,...,...,...,...,...,...
112,334.339996,339.040009,332.809998,337.339996,26003800,337.339996,0.0,MSFT,2023-06-14
113,337.480011,349.839996,337.200012,348.100006,38899100,348.100006,0.0,MSFT,2023-06-15
114,351.320007,351.470001,341.950012,342.329987,46533600,342.329987,0.0,MSFT,2023-06-16
115,339.309998,342.079987,335.859985,338.049988,26375400,338.049988,0.0,MSFT,2023-06-20


In [ ]:
import plotly.express as px

px.line(history_df, x="date", y="adjclose", title=f"Adjusted Closing Prices for {symbol}")

In [ ]:
#px.bar(history_df, x="date", y="dividends", title=f"Dividends for {symbol}")

### Multiple Stocks

Let's choose around ten stocks of interest. Feel free to update the list of ticker symbols below.

In [ ]:
# setup cell (run and leave as is)

from yahooquery import Ticker

symbols = ["AAPL", "GOOGL", "META", "MSFT", "NFLX", "AMZN", "NVDA",
           "BAC", "JPM"
]
companies = Ticker(symbols)
print(type(companies))

<class 'yahooquery.ticker.Ticker'>


Let's grab the historical prices for them all. We know we will get a somewhat complicated multi-indexed dataframe back.

In [ ]:
companies.history()

open        high         low       close     volume  \
symbol date                                                                    
AAPL   2023-01-03  130.279999  130.899994  124.169998  125.070000  112117500   
       2023-01-04  126.889999  128.660004  125.080002  126.360001   89113600   
       2023-01-05  127.129997  127.769997  124.760002  125.019997   80962700   
       2023-01-06  126.010002  130.289993  124.889999  129.619995   87754700   
       2023-01-09  130.470001  133.410004  129.889999  130.149994   70790800   
...                       ...         ...         ...         ...        ...   
JPM    2023-06-14  143.000000  143.610001  140.699997  141.490005    9068700   
       2023-06-15  140.589996  143.399994  140.029999  143.089996    9343100   
       2023-06-16  143.050003  143.970001  142.500000  143.259995   13812100   
       2023-06-20  142.610001  143.000000  141.039993  142.529999    8428600   
       2023-06-21  142.899994  144.020004  141.690002  142.320007    9923000   

                     adjclose  dividends  
symbol date                               
AAPL   2023-01-03  124.706833        0.0  
       2023-01-04  125.993095        0.0  
       2023-01-05  124.656975        0.0  
       2023-01-06  129.243622        0.0  
       2023-01-09  129.772079        0.0  
...                       ...        ...  
JPM    2023-06-14  141.490005        0.0  
       2023-06-15  143.089996        0.0  
       2023-06-16  143.259995        0.0  
       2023-06-20  142.529999        0.0  
       2023-06-21  142.320007        0.0  

[1053 rows x 7 columns]

In [ ]:
from pandas import to_datetime

histories_df = companies.history()
#histories_df.index.get_level_values(0)
#histories_df.index.get_level_values(1)

histories_df["symbol"] = histories_df.index.get_level_values(0)
histories_df["date"] = to_datetime(histories_df.index.get_level_values(1)).date
histories_df.reset_index(drop=True, inplace=True) # see: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
print(len(histories_df)) #> 1053 rows
histories_df[["date", "symbol", "adjclose"]]

1053


,date,symbol,adjclose
0,2023-01-03,AAPL,124.706833
1,2023-01-04,AAPL,125.993095
2,2023-01-05,AAPL,124.656975
3,2023-01-06,AAPL,129.243622
4,2023-01-09,AAPL,129.772079
...,...,...,...
1048,2023-06-14,JPM,141.490005
1049,2023-06-15,JPM,143.089996
1050,2023-06-16,JPM,143.259995
1051,2023-06-20,JPM,142.529999


In [ ]:
# quick check for null values (because some stocks may have different history lengths)
# https://stackoverflow.com/questions/26266362/how-do-i-count-the-nan-values-in-a-column-in-pandas-dataframe
histories_df["adjclose"].isnull().sum()  #> 0 ok looks good. can proceed without concern for nulls

0

#### Grouping and Aggregations

By default, the dataset is formatted as a **row per date per symbol**, with a **single column of (adjusted closing) prices**.

We can use this kind of data format to do easy grouping and aggregation, for example, calculating the average price for each stock, as well as the volatility / risk level (standard deviation of prices) for each stock.

> Beta and standard deviation are two ways to describe market volatility. Beta measures the fund's volatility in comparison to other funds, whereas standard deviation measures the fluctuation in the fund's share price over time. - [source](https://www.tutorialspoint.com/difference-between-beta-and-standard-deviation)


In [ ]:
print("AVERAGE PRICE PER STOCK:")
histories_df.groupby("symbol")["adjclose"].mean().sort_values(ascending=False)

AVERAGE PRICE PER STOCK:


symbol
NFLX     345.091368
MSFT     280.546800
NVDA     265.622497
META     203.974273
AAPL     159.034452
JPM      136.477879
GOOGL    104.643932
AMZN     103.991282
BAC       30.632444
Name: adjclose, dtype: float64

In [ ]:
print("VOLATILITY (STANDARD DEVIATION) PER STOCK:")
histories_df.groupby("symbol")["adjclose"].std().sort_values(ascending=False)

VOLATILITY (STANDARD DEVIATION) PER STOCK:


symbol
NVDA     71.178386
META     43.533557
NFLX     35.039349
MSFT     33.262708
AAPL     15.253099
GOOGL    11.706819
AMZN     10.791435
JPM       4.979865
BAC       3.016316
Name: adjclose, dtype: float64

#### Pivot Operations


However, for charting purposes, we will want to construct a differently formatted dataset with a **row per date**, and a **column of adjusted closing prices for each symbol**.

If you are not sure how to achieve this, you could try an internet search for something like "pandas single column to multiple columns for different values", which will yeild an [SO post like this](https://stackoverflow.com/questions/58134672/converting-single-column-to-multiple-columns-based-on-unique-values) talking about a pivot-based approach.

Now let's try to adapt the approach from this post, while cross referencing our notes about pivot operations, as well as the [`pivot` function docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html), as necessary.



In [ ]:
# using the pivot, we say we want a column for each symbol,
# and use the adjusted closing prices as the values
# and the dates as the rows
prices_pivot = histories_df.pivot(columns="symbol", values="adjclose", index="date")
print(len(prices_pivot))
# adding a column of dates for charting later
prices_pivot["date"] = prices_pivot.index
prices_pivot.reset_index(inplace=True, drop=True)
prices_pivot

117


symbol,AAPL,AMZN,BAC,GOOGL,JPM,META,MSFT,NFLX,NVDA,date
0,124.706833,85.820000,33.030487,89.120003,133.084778,124.739998,238.460144,294.950012,143.110870,2023-01-03
1,125.993095,85.139999,33.651470,88.080002,134.325806,127.370003,228.029129,309.410004,147.449707,2023-01-04
2,124.656975,83.120003,33.582474,86.199997,134.296051,126.940002,221.270844,309.700012,142.611023,2023-01-05
3,129.243622,86.080002,33.917606,87.339996,136.865875,130.020004,223.878601,315.549988,148.549393,2023-01-06
4,129.772079,87.360001,33.405048,88.019997,136.300308,129.470001,226.058380,315.170013,156.237289,2023-01-09
...,...,...,...,...,...,...,...,...,...,...
112,183.949997,126.419998,29.120001,123.669998,141.490005,273.350006,337.339996,440.859985,429.970001,2023-06-14
113,186.009995,127.110001,29.370001,125.089996,143.089996,281.829987,348.100006,445.269989,426.529999,2023-06-15
114,184.919998,125.489998,29.190001,123.529999,143.259995,281.000000,342.329987,431.959991,426.920013,2023-06-16
115,185.009995,125.779999,28.870001,123.099998,142.529999,284.329987,338.049988,434.700012,438.079987,2023-06-20


In [ ]:
import plotly.express as px

#px.line(prices_pivot, x="date", y="AAPL")
#px.line(prices_pivot, x="date", y=["AAPL", "AMZN"])
px.line(prices_pivot, x="date", y=symbols, title="Stock Prices (Raw Values)")

#### Growth Calculations

The prices all have different ranges, and plotting the raw data on a graph does not allow us to see which have the highest relative growth.

Instead, after getting the raw stock prices, we can convert them to a relative scale (for example calculating percentage growth, or cumulative growth). Then we can plot these normalized prices to provide us with proper ability to compare.

https://www.codingfinance.com/post/2018-04-03-calc-returns-py/

In other words, let's treat each stock's price on the first day as a value of 1, and then plot the growth in relative terms from there.

In [ ]:
# knowing we are going to perform some shift-based methods,
# we first ensure data is sorted, for good measure
prices_pivot.sort_values(by="date", ascending=True, inplace=True)
prices_pivot

symbol,AAPL,AMZN,BAC,GOOGL,JPM,META,MSFT,NFLX,NVDA,date
0,124.706833,85.820000,33.030487,89.120003,133.084778,124.739998,238.460144,294.950012,143.110870,2023-01-03
1,125.993095,85.139999,33.651470,88.080002,134.325806,127.370003,228.029129,309.410004,147.449707,2023-01-04
2,124.656975,83.120003,33.582474,86.199997,134.296051,126.940002,221.270844,309.700012,142.611023,2023-01-05
3,129.243622,86.080002,33.917606,87.339996,136.865875,130.020004,223.878601,315.549988,148.549393,2023-01-06
4,129.772079,87.360001,33.405048,88.019997,136.300308,129.470001,226.058380,315.170013,156.237289,2023-01-09
...,...,...,...,...,...,...,...,...,...,...
112,183.949997,126.419998,29.120001,123.669998,141.490005,273.350006,337.339996,440.859985,429.970001,2023-06-14
113,186.009995,127.110001,29.370001,125.089996,143.089996,281.829987,348.100006,445.269989,426.529999,2023-06-15
114,184.919998,125.489998,29.190001,123.529999,143.259995,281.000000,342.329987,431.959991,426.920013,2023-06-16
115,185.009995,125.779999,28.870001,123.099998,142.529999,284.329987,338.049988,434.700012,438.079987,2023-06-20


Illustration of methods:

In [ ]:
#prices_pivot["AAPL"].pct_change(periods=1)

In [ ]:
#prices_pivot["AAPL"].pct_change(periods=1) + 1

In [ ]:
(prices_pivot["AAPL"].pct_change(periods=1) + 1).cumprod()

0           NaN
1      1.010314
2      0.999600
3      1.036380
4      1.040617
         ...   
112    1.475059
113    1.491578
114    1.482838
115    1.483559
116    1.475140
Name: AAPL, Length: 117, dtype: float64

In [ ]:
# ignoring some warnings that these operations will throw
import warnings
warnings.filterwarnings("ignore")


returns_df = prices_pivot.copy()

for symbol in symbols:
    growth_colname = f"{symbol}_growth"
    returns_df[growth_colname] = (returns_df[symbol].pct_change(periods=1) + 1).cumprod()
    returns_df[growth_colname][0] = 1 # set NaN to 1

returns_df.reset_index(inplace=True, drop=True)
returns_df[["AAPL", "AAPL_growth"]]

symbol,AAPL,AAPL_growth
0,124.706833,1.000000
1,125.993095,1.010314
2,124.656975,0.999600
3,129.243622,1.036380
4,129.772079,1.040617
...,...,...
112,183.949997,1.475059
113,186.009995,1.491578
114,184.919998,1.482838
115,185.009995,1.483559


It is always important to check our work. Let's do a quick check to see if our cumulative growth matches our expectations. Here we see the final growth value matches our check.

In [ ]:
beginning = returns_df.iloc[0]["AAPL"]
ending = returns_df.iloc[-1]["AAPL"]
growth_check = (ending - beginning) / beginning
print(growth_check)


print(returns_df.iloc[-1]["AAPL_growth"])
print(returns_df.iloc[-1]["AAPL_growth"] - 1) # let's subtract one to undo our previous operation

0.47513975342805337
1.4751397534280528
0.4751397534280528


OK great, now we can chart the data on a single plot, and also compare the growth across each company.

In [ ]:
growth_cols = [f"{symbol}_growth" for symbol in symbols]
print(growth_cols)

['AAPL_growth', 'GOOGL_growth', 'META_growth', 'MSFT_growth', 'NFLX_growth', 'AMZN_growth', 'NVDA_growth', 'BAC_growth', 'JPM_growth']


In [ ]:
import plotly.express as px

px.line(returns_df, x="date", y=growth_cols, title="Stock Prices (Cumulative Growth)")

Now it is easy to tell the story of which company has the highest growth!

In [ ]:
returns_df[growth_cols].iloc[-1].sort_values(ascending=False)

symbol
NVDA_growth     3.007808
META_growth     2.257816
AAPL_growth     1.475140
AMZN_growth     1.454556
NFLX_growth     1.439057
MSFT_growth     1.398808
GOOGL_growth    1.352671
JPM_growth      1.069394
BAC_growth      0.864958
Name: 116, dtype: float64